In [1]:
# tensorflowがgpuを使っているか確認する
import tensorflow as tf
print(tf.test.is_built_with_cuda())

True


In [1]:
from ray.rllib.algorithms.dreamerv3 import DreamerV3Config
from envs.gym_env import MyEnv
from ray import train, tune, air
from ray.rllib.algorithms.sac import SACConfig
from ray.rllib.algorithms.algorithm import Algorithm
from ray.air.integrations.wandb import WandbLoggerCallback

def env_creator(env_config):
    return MyEnv()

tune.register_env("my_env", env_creator)

# config = (  # 1. Configure the algorithm,
#     DreamerV3Config()
#     .environment('my_env')
#     .training(
#         model_size="XS", # XS, S, M, L, XL
#     )
# )

# https://docs.ray.io/en/latest/rllib/rllib-training.html#specifying-resources

model_dict = {"conv_filters": [
            [16, [8, 8], 4],
            [32, [4, 4], 4],
            [64, [4, 4], 2],
            [128, [4, 4], 2]
            ]}

#config = SACConfig().environment(MyEnv, disable_env_checking=True, render_env=True).rollouts(num_rollout_workers=5).training(train_batch_size=64, model=model_dict).framework(framework="torch").resources(num_gpus=1)
config = DreamerV3Config().environment(env="my_env", disable_env_checking=True, render_env=True).rollouts(num_rollout_workers=10, remote_worker_envs=False).training(model=model_dict).resources(num_gpus=1)

tuner = tune.Tuner(
    "DreamerV3",  # 2. Specify the algorithm name
    run_config=train.RunConfig(
        stop={"episode_reward_mean": 30
              #"timesteps_total": 100000,
              },  # 学習の終了条件
        callbacks=[WandbLoggerCallback(project="Wandb_nearme")]  # コメントアウトを外すとwandbにログが送信される(https://docs.wandb.ai/quickstart)
    ),
    param_space=config,
)
results = tuner.fit()
best_result = results.get_best_result(metric="episode_reward_mean", mode="max")
best_checkpoint = best_result.checkpoint

algo = Algorithm.from_checkpoint(best_checkpoint)

env = MyEnv()
episode_reward = 0
terminated = truncated = False

obs, info = env.reset()
while not terminated and not truncated:
    env.render()
    action = algo.compute_single_action(obs)
    obs, reward, terminated, truncated, info = env.step(action)
    episode_reward += reward
print(f"Episode reward :{episode_reward}")

2024-02-22 01:04:08,599	INFO wandb.py:307 -- Already logged into W&B.
(pid=13244) c:\Users\alaleh\anaconda3\envs\rl_env\lib\site-packages\gymnasium\utils\passive_env_checker.py:135: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: float64
(pid=13244)   logger.warn(
(pid=13244) c:\Users\alaleh\anaconda3\envs\rl_env\lib\site-packages\gymnasium\utils\passive_env_checker.py:159: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
(pid=13244)   logger.warn(f"{pre} is not within the observation space.")
(pid=44076) c:\Users\alaleh\anaconda3\envs\rl_env\lib\site-packages\gymnasium\utils\passive_env_checker.py:135: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: float64
(pid=44076)   logger.warn(
(pid=44076) c:\Users\alaleh\anaconda3\envs\rl_env\lib\site-packages\gymnasium\utils\passive_env_checker.py:159: Us

(raylet) The node with node id: 782d651f52250a124dc56ef675356fe8fcba648928cd226bab51ae7a and address: 127.0.0.1 and node name: 127.0.0.1 has been marked dead because the detector has missed too many heartbeats from it. This can happen when a 	(1) raylet crashes unexpectedly (OOM, preempted node, etc.) 
	(2) raylet has lagging heartbeats due to slow network or busy workload.


KeyError: DreamerV3_my_env_d84dc_00000